<a href="https://colab.research.google.com/github/Kogo0813/Windroad8/blob/main/TPUver_windpredict_data_from_s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!echo $COLAB_TPU_ADDR #코랩에서 런타임->런타임 유형 변경 -> TPU로 변경

10.26.2.114:8470


In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from osgeo import gdal

# TPU gRPC 접근 URI
TPU_PATH = f"grpc://{os.environ['COLAB_TPU_ADDR']}"
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_PATH)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [6]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive
%cd MyDrive
%cd data_from_s

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/data_from_s


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Dense, Flatten
import tensorflow.keras.backend as K
from keras.utils import get_custom_objects

with strategy.scope():
  def r2_metric(y_true, y_pred):
      SS_res =  K.sum(K.square(y_true - y_pred))
      SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
      return (1 - SS_res/(SS_tot + K.epsilon()))

  get_custom_objects().update({'r2_metric': r2_metric})
  width = 75
  height = 60
  channels = 7
  model = Sequential([
     keras.Input(shape=(None,height,width,channels)),
      layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True),
      layers.BatchNormalization(),
     layers.ConvLSTM2D(filters=32, kernel_size=(3, 3), padding="same", return_sequences=True),
      layers.BatchNormalization(),
     layers.ConvLSTM2D(filters=16, kernel_size=(3, 3), padding="same", return_sequences=False),
     layers.BatchNormalization(),
     layers.Conv2D(filters=2,kernel_size=(3,3),activation="sigmoid",padding="same"),
  ])
  # 모델 컴파일
  model.compile(optimizer='adam', loss='mse',metrics='r2_metric')
  model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, None, 60, 75, 32)  45056     
                                                                 
 batch_normalization (BatchN  (None, None, 60, 75, 32)  128      
 ormalization)                                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, None, 60, 75, 32)  73856     
                                                                 
 batch_normalization_1 (Batc  (None, None, 60, 75, 32)  128      
 hNormalization)                                                 
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 60, 75, 16)        27712     
                                                                 
 batch_normalization_2 (Batc  (None, 60, 75, 16)       6

In [9]:
from skimage.measure import block_reduce ## 필독 : 1200 * 1500의 이미지 데이터가 코랩에서 돌리기에 너무 큰 관계로 60*75로 다운샘플링해서 사용하였습니다.
from sklearn.preprocessing import MinMaxScaler # 입력 정규화 Min-Max 스케일링 사용


X = []
Y = []
for i in range(1, 12):
  for h in range(1, 4):
    for s in range(0,6):
      for d in range(0, 360, 60):
        height = np.full((60,75),(h-1)/2) #지역풍 높이는 1부터 3까지 있으므로 이렇게 정규화시킴
        speed = np.full((60,75), s/5) #속도도 마찬가지 0부터 5까지
        direction = np.full((60,75),d/300) # 방향 0부터 300까지
        uz_zero = np.full((60,75),0)
        vz_zero = np.full((60,75),0)
        scaler = MinMaxScaler()
        landuse = gdal.Open("입력데이터/"+"Landuse_"+"{:006d}".format(i)+".tif")
        landuse = np.array(landuse.GetRasterBand(1).ReadAsArray())
        terrain = gdal.Open("입력데이터/"+"Terrain"+"{:006d}".format(i)+".tif")
        terrain = np.array(terrain.GetRasterBand(1).ReadAsArray())
        landuse = block_reduce(landuse, block_size=(20, 20), func=np.mean)
        terrain = block_reduce(terrain, block_size=(20, 20), func=np.mean)
        terrain = scaler.fit_transform(terrain)
        landuse = scaler.fit_transform(landuse)
        stacked_layer_input = []
        stacked_layer_input.append(np.dstack((uz_zero,vz_zero,landuse,terrain,direction,speed,height)))
        for j in range(360, 3960, 360):
            uz = np.loadtxt("0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)+'_uz00'+"{:04d}".format(j)+'.dw', skiprows = 8, dtype = 'int', encoding='latin-1')
            vz = np.loadtxt("0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)+'_vz00'+"{:04d}".format(j)+'.dw', skiprows = 8, dtype = 'int', encoding='latin-1')
            uz = block_reduce(uz, block_size=(20, 20), func=np.mean)
            vz = block_reduce(vz, block_size=(20, 20), func=np.mean)
            uz = scaler.fit_transform(uz)
            vz = scaler.fit_transform(vz)
            if j == 3600:
              stacked_layer_output = np.dstack((uz,vz))
            else:
              stacked_layer_input.append(np.dstack((uz,vz,landuse,terrain,direction,speed,height)))
        X.append(stacked_layer_input)
        Y.append(stacked_layer_output)


AttributeError: ignored

In [ ]:
X = np.array(X)
Y = np.array(Y)
X.shape

(36, 10, 60, 75, 7)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit(x=X, y=Y, validation_split=0.2, epochs=10, callbacks=[early_stop])

Epoch 1/10
1/1 [==============================] - 119s 119s/step - loss: 119.8532 - r2_metric: -0.0058 - val_loss: 124.8628 - val_r2_metric: -0.0015
Epoch 2/10
1/1 [==============================] - 87s 87s/step - loss: 116.6796 - r2_metric: 0.0208 - val_loss: 124.6428 - val_r2_metric: 2.6369e-04
Epoch 3/10
1/1 [==============================] - 91s 91s/step - loss: 115.2920 - r2_metric: 0.0324 - val_loss: 124.4411 - val_r2_metric: 0.0019
Epoch 4/10
1/1 [==============================] - 89s 89s/step - loss: 114.2634 - r2_metric: 0.0411 - val_loss: 124.2992 - val_r2_metric: 0.0030
Epoch 5/10
1/1 [==============================] - 94s 94s/step - loss: 113.6315 - r2_metric: 0.0464 - val_loss: 124.1928 - val_r2_metric: 0.0039
Epoch 6/10
1/1 [==============================] - 93s 93s/step - loss: 113.2830 - r2_metric: 0.0493 - val_loss: 124.0091 - val_r2_metric: 0.0053
Epoch 7/10
1/1 [==============================] - 87s 87s/step - loss: 112.9874 - r2_metric: 0.0518 - val_loss: 123.8017 -